TEST ML_composer 


In [1]:
import os
print(os.getcwd())
#os.chdir("F:/project/sugarcane_disease/")
os.chdir("O:/project/sugarcane_disease/")
os.getcwd()
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

O:\project\ML_composer


In [2]:
from GS_composer import *

Use tensorflow backend keras module


In [3]:
from ClassModel import *


In [4]:
from Functions import *

In [7]:
class p_args:
    
    def __init__(self):
        
        self.ped = "sugarcane_disease"
        self.pheno = "sugarcane_disease.pheno"
        self.mpheno = 1
        self.index = "sugarcane_disease.index"
        self.model = "Numeric CNN"
        self.load = None
        self.trait = "trait"
        self.output = "./results"
        self.round = 10
        self.plot = True
        self.epoch = 50
        self.rank =True
        self.lr = 0.0001
        
args = p_args()

In [8]:
config = None


In [9]:
if args.output[0] == "/":
    locat = '/' + args.output.strip('/') + '/'
else:
    locat = args.output.strip('/') + '/'
if not os.path.exists(locat):
    os.mkdir(locat)

In [13]:
composer = ML_composer()
composer._model
composer.silence_mode = 2

In [14]:
composer.get_data(config,args)

1    939
3    940
4    940
2    940
5    943
Name: 2, dtype: int64
Get genotype shape: (4702, 26086)
   7   8   9   10  11  12  13  14  15
1   0   2   0   1   2   1   0   1   2
2   1   2   1   1   2   1   0   1   1
3   1   2   0   0   1   0   1   1   2
4   1   2   0   0   1   0   1   1   2
5   0   2   1   1   2   0   2   1   1
6   0   2   0   1   2   1   0   1   2
7   1   2   1   1   2   1   2   1   2
8   0   2   0   1   1   0   1   0   2
9   1   2   1   1   2   1   2   1   2


In [15]:
composer.prepare_model()

In [16]:
index_ref = composer.prepare_cross_validate()

In [17]:
print("Cross-validate: {}".format(index_ref[0]))

Cross-validate: ([2, 3, 4, 5], [1])


In [18]:
train_idx,valid_idx = index_ref[0]

In [19]:
composer.prepare_training(train_idx,valid_idx)
composer.batchSize = 64

Overall population: 4702
759 individuals need to be removed due to the miss phenotype
Filtered population: 3943
USE Numeric CNN MODEL as training method
The transformed SNP shape: (3047, 26086, 1)
USE Numeric CNN MODEL as training method
The transformed SNP shape: (896, 26086, 1)


In [21]:
composer._raw_data["GENO"].iloc[:,6:].describe()

,6,7,8,9,10,11,12,13,14,15,...,26082,26083,26084,26085,26086,26087,26088,26089,26090,26091
count,4702.000000,4702.000000,4702.000000,4702.000000,4702.000000,4702.000000,4702.000000,4702.000000,4702.000000,4702.000000,...,4702.000000,4702.000000,4702.000000,4702.000000,4702.000000,4702.000000,4702.000000,4702.000000,4702.000000,4702.000000
mean,1.675457,0.295194,1.581455,0.578052,0.509783,1.692471,0.576563,0.770949,0.608252,1.778179,...,0.486176,1.293492,1.614207,0.756061,0.379200,1.352616,1.448745,1.698426,0.374309,1.732880
std,0.473224,0.471766,0.580163,0.624409,0.613791,0.500439,0.630694,0.686006,0.660426,0.428123,...,0.521930,0.633561,0.497638,0.666646,0.542373,0.590891,0.611725,0.479392,0.532144,0.474066
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000,...,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
50%,2.000000,0.000000,2.000000,1.000000,0.000000,2.000000,0.000000,1.000000,1.000000,2.000000,...,0.000000,1.000000,2.000000,1.000000,0.000000,1.000000,2.000000,2.000000,0.000000,2.000000
75%,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000,...,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000,2.000000,2.000000,1.000000,2.000000
max,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


In [15]:
composer.compose(train_idx,valid_idx)

Train status:
Epochs:  50
Repeat(Round):  10


ValueError: The first argument to `Layer.call` must always be passed.

In [ ]:
#import tensorflow as tf 

if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

In [ ]:
1.5//1

In [ ]:
index_ref = composer.prepare_cross_validate()
i = 1
for train_idx,valid_idx in index_ref:
    print("Cross-validate: {}".format(i))
    composer.prepare_training(train_idx,valid_idx)
    composer.compose(train_idx,valid_idx)
    i+=1